<a href="https://colab.research.google.com/github/zakarka2006/famcs2024/blob/main/famcs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone https://github.com/zakarka2006/famcs2024.git

fatal: destination path 'famcs2024' already exists and is not an empty directory.


In [3]:
!pip install opencv-python pytesseract

# Letters №1


In [34]:
import cv2
import numpy as np
from collections import Counter
from PIL import Image, ImageEnhance

Image.MAX_IMAGE_PIXELS = 100000000
def maximize_contrast(image_path):
    image = Image.open(image_path)
    image = image.convert("L")
    enhancer = ImageEnhance.Contrast(image)
    max_contrast_image = enhancer.enhance(10)
    max_contrast_image.save(f"contrast_{image_path}")

def preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, binary_image = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    return binary_image

def find_contours(binary_image):
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def create_letter_templates():
    templates = {}
    for letter in "ABCDEFGHIJKLMNOPQRSTUVWXYZ":
        template_path = f"./letters/{letter}.png"
        template_image = preprocess_image(template_path)
        contours = find_contours(template_image)
        if contours:  # Only add templates with valid contours
            templates[letter] = contours
    return templates

def match_letters(image, templates):
    detected_letters = []
    contours = find_contours(image)
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        letter_image = image[y:y+h, x:x+w]
        max_match = float("inf")
        matched_letter = ""
        for letter, template_contours in templates.items():
            for template_contour in template_contours:
                match = cv2.matchShapes(template_contour, contour, cv2.CONTOURS_MATCH_I3, 0.0)
                if match < max_match:
                    max_match = match
                    matched_letter = letter
        detected_letters.append(matched_letter)
    return detected_letters

maximize_contrast("letters.png")
binary_image = preprocess_image("contrast_letters.png")
templates = create_letter_templates()
detected_letters = match_letters(binary_image, templates)

letter_count = Counter(detected_letters)
print(letter_count)

counter = 0
for letter in letter_count:
    if (letter.lower() == letter):
        continue
    counter += letter_count[letter]
    print(letter, letter_count[letter])

print(counter)


Counter({'C': 935, 'V': 888, 'Y': 857, 'I': 797, 'U': 763, 'N': 762, 'Z': 744, 'H': 740, 'M': 727, 'R': 657, 'A': 651, 'S': 630, 'B': 565, 'K': 562, 'G': 557, 'W': 526, 'D': 498, 'O': 371, 'J': 322, 'L': 188, 'Q': 188, 'F': 183, 'T': 169, 'P': 86, 'X': 70, 'E': 20})
F 183
Y 857
I 797
K 562
H 740
L 188
J 322
A 651
D 498
W 526
Z 744
U 763
N 762
R 657
V 888
M 727
B 565
X 70
T 169
C 935
O 371
G 557
Q 188
S 630
P 86
E 20
13456
